In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
root_dir = Path.cwd().parents[1]
if not root_dir in sys.path: sys.path.insert(0,str(root_dir))
print(root_dir)

/home/wollex/Data/Science/WolfGroup/DarkMatter/Programme


In [ ]:
import pprint

import numpy as np
from matplotlib import pyplot as plt
from inference.BayesModel import BayesModel
from inference.HierarchicalBayesModel import prior_structure, norm_ppf, halfnorm_ppf
from inference.network import Network
from empirical.create_surrogate_data import SurrogateData

from inference.HierarchicalBayesModel.NestedSamplingMethods import (
    run_sampling,
    plot_results,
)

In [ ]:
def create_data_from_params(
    ground_truth_params, N=1000, T=1200, n_animals=1, plot=False
):
    net = Network(**{key: ground_truth_params[key] for key in ["eps", "eta"]})
    net = Network()
    for p, pop in enumerate(ground_truth_params["distr"]):
        net.register_population(
            p, **{key: pop[key] for key in ["nu_bar", "alpha_0", "J0"]}
        )

        for s, syn in enumerate(pop["synapses"]):
            net.populations[p].register_synapse(
                s, **{key: syn[key] for key in ["tau_I"]}
            )

    net.solve_selfcon()

    ref_values = {
        "distr": net.export_distr(),
        # "distr": [
        #     distr_params(gamma=2.1, delta=4., nu_max=25.0,kappa=4.),
        #     distr_params(gamma=1.5, delta=4.5, nu_max=25.0,kappa=1.),
        #     # distr_params(gamma=1.9, delta=3.5, nu_max=25.0,kappa=1.),
        # ],
        "join": False,
        "N": N,
        "T": T,
    }

    SD = SurrogateData(N=ref_values["N"], T=ref_values["T"], n_animals=n_animals)
    SD.artificial_data(ref_values, plot=False)
    if plot:
        SD.plot_rates()

    return SD, net

In [18]:
def define_biological_prior(prior_shape=(1,)):
    def bounded_flat(x, low, high):
        return x * (high - low) + low

    prior = {}
    prior["eps"] = prior_structure(
        bounded_flat, low=0.0, high=np.sqrt(1.0 / 2), label="$\\epsilon$"
    )
    prior["eta"] = prior_structure(bounded_flat, low=0.0, high=1.0, label="$\\eta$")

    for p in range(2):
        prior[f"nu_bar_pop{p}"] = prior_structure(
            halfnorm_ppf,
            loc=0.0,
            scale=5.0,
            # norm_ppf,
            # mean=prior_structure(halfnorm_ppf, loc=0.0, scale=5.0),
            # sigma=prior_structure(halfnorm_ppf, loc=0.0, scale=1.),
            # shape=(n_conditions,) if p==1 else (1,),
            shape=prior_shape,
            label="$\\bar{\\nu}_" + str(p) + "$",
        )

        prior[f"alpha_0_pop{p}"] = prior_structure(
            halfnorm_ppf, loc=0.0, scale=0.05, label="$\\alpha_0$"
        )

        prior[f"tau_I_s0_pop{p}"] = prior_structure(
            # None,value=[0.03 if p==0 else 0.005],
            halfnorm_ppf,
            loc=0.0,
            scale=0.02,
            label="$\\tau_{I_" + str(p) + "}$",
        )
    return prior

def define_prior(prior_shape=(1,)):
    prior = {}
    p=0
    prior[f"gamma_pop{p}"] = prior_structure(
        halfnorm_ppf,
        loc=1.0,
        scale=0.5,
        # norm_ppf,
        # mean=prior_structure(halfnorm_ppf, loc=0.0, scale=5.0),
        # sigma=prior_structure(halfnorm_ppf, loc=0.0, scale=1.),
        # shape=(n_conditions,) if p==1 else (1,),
        shape=prior_shape,
        label="$\\gamma_" + str(p) + "$",
    )

    prior[f"delta_pop{p}"] = prior_structure(
        norm_ppf, loc=6.0, scale=2., label="$\\delta$"
    )

    prior[f"nu_max_pop{p}"] = prior_structure(
        # None,value=[0.03 if p==0 else 0.005],
        halfnorm_ppf,
        loc=0.0,
        scale=30.,
        label="$\\nu_{max_" + str(p) + "}$",
    )
    return prior

In [19]:
def prepare_model(event_counts, prior, T=120.0, biological=False):
    BM = BayesModel()  # logLevel=logging.WARNING)
    BM.prepare_data(event_counts, T=T)
    BM.set_priors(prior)
    prior_transformation = BM.set_prior_transform(vectorized=False)
    loglikelihood = BM.set_logl(
        vectorized=False,
        correct_N=0,
        # bias_to_expected_max=0,
        # bias_to_mean=0,
        biological=biological,
    )
    return prior_transformation, loglikelihood, BM

In [20]:
def build_truths(ground_truth_params, conditions=None):
    truths = {}
    for key in ["eps", "eta"]:
        if key in ground_truth_params:
            truths[key] = ground_truth_params[key]
    for p, pop in enumerate(ground_truth_params["distr"]):
        for key in ["nu_bar", "alpha_0"]:
            truths[f"{key}_pop{p}"] = pop[key]

        for s, syn in enumerate(pop["synapses"]):
            for key in ["tau_I"]:
                truths[f"{key}_s{s}_pop{p}"] = syn[key]
    if conditions:
        for key, val in conditions.items():
            truths[key] = val  # for all conditions
    return truths

In [21]:
ground_truth_params = {
    # "eps": 0.6,
    # "eta": 0.4,
    "distr": [
        # {
        #     "nu_bar": 2.0,
        #     "alpha_0": 0.02,
        #     "J0": -1.0,
        #     "synapses": [{"tau_I": 0.03}],
        # },
        {
            "nu_bar": 1.0,
            "alpha_0": 0.04,
            "J0": 1.0,
            "synapses": [{"tau_I": 0.005}],
        },
    ],
}

In [22]:
truths = build_truths(ground_truth_params)

N = 100
T = 60
n_animals = 1
event_counts = np.zeros((n_animals, 2, N))
# for i, nu in enumerate(conditions):
# gtp["distr"][1]["nu_bar"] = nu
SD, net = create_data_from_params(
    ground_truth_params, N=N, T=T, n_animals=n_animals, plot=True
)
# event_counts[:,i,...] = SD.event_counts

prior = define_biological_prior(SD.event_counts.shape[:-2])
prior_transformation, loglikelihood, BM = prepare_model(
    SD.event_counts, prior, T, biological=True
)
results, sampler = run_sampling(
    prior_transformation,
    loglikelihood,
    BM.parameter_names_all,
    mode="dynesty",
    nP=12,
    n_live=100,
)

plot_results(BM, results, mode="dynesty", truths=truths)

KeyError: 'eps'